# Install packages
*inspired from Unsloth's Deepseek-OCR fine-tuning notebook*

In [1]:
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install jiwer
!pip install einops addict easydict
!pip install verovio
!pip install faker
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 16.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 8.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling multiprocess-0.70.18:
      Successfully uninstalled multiprocess-0.70.18
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
  Attempting uninstall: datasets
    Found existing installat

## Download Deepseek-OCR from Unsloth

In [2]:
from huggingface_hub import snapshot_download
snapshot_download("unsloth/DeepSeek-OCR", local_dir = "deepseek_ocr")

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

assets/fig1.png:   0%|          | 0.00/396k [00:00<?, ?B/s]

assets/show3.jpg:   0%|          | 0.00/247k [00:00<?, ?B/s]

assets/show1.jpg:   0%|          | 0.00/117k [00:00<?, ?B/s]

assets/show2.jpg:   0%|          | 0.00/216k [00:00<?, ?B/s]

...

modeling_deepseekv2.py: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

'/kaggle/working/deepseek_ocr'

### HF Login for Kaggle

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("HF_TOKEN")
login(token=token)

### Hotfix (Transformers >= 4.46 compatibility)

In [4]:
import transformers.models.llama.modeling_llama
if not hasattr(transformers.models.llama.modeling_llama, "LlamaFlashAttention2"):
    print(">>> Monkeypatching LlamaFlashAttention2 for DeepSeek-OCR compatibility...")
    transformers.models.llama.modeling_llama.LlamaFlashAttention2 = transformers.models.llama.modeling_llama.LlamaAttention

2026-01-20 07:10:52.349191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768893052.778274      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768893052.913393      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768893053.982775      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768893053.982811      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768893053.982814      55 computation_placer.cc:177] computation placer alr

>>> Monkeypatching LlamaFlashAttention2 for DeepSeek-OCR compatibility...


# Imports

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    AutoModel,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
from transformers.modeling_outputs import CausalLMOutputWithPast
import torchvision.transforms as T
import warnings
import gc
from PIL import Image, ImageDraw, ImageFont
import os
import json
import random
from faker import Faker
from peft import get_peft_model, LoraConfig
import types

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Projector MLP

In [33]:
class DeepSeekOCRToGOTProjector(nn.Module):
    def __init__(self, encoder_dim, decoder_dim=1024):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(encoder_dim, decoder_dim * 2),
            nn.GELU(),
            nn.Linear(decoder_dim * 2, decoder_dim),
            nn.LayerNorm(decoder_dim)
        )

    def forward(self, x):
        return self.net(x)

# Fusion Model

In [34]:
class DeepSeekGOTFusion(nn.Module):
    def __init__(self, deepseek_ocr_path, got_path, tokenizer, use_lora=True):
        super().__init__()
        warnings.filterwarnings("ignore")
        frozen_dtype = torch.float16
        self.vision_dtype = frozen_dtype

        print(f">>> Loading DeepSeek-OCR Encoder...")
        tmp_ds = AutoModel.from_pretrained(
            deepseek_ocr_path,
            trust_remote_code=True,
            device_map="cpu",
            torch_dtype=frozen_dtype,
            low_cpu_mem_usage=True
        )

        if hasattr(tmp_ds, "model"):
            base_model = tmp_ds.model
        else:
            base_model = tmp_ds

        if hasattr(base_model, "deep_encoder"):
            self.vision_tower = base_model.deep_encoder
        elif hasattr(base_model, "vision_model"):
            self.vision_tower = base_model.vision_model
        else:
            self.vision_tower = base_model

        self.vision_dim = 1024
        if hasattr(self.vision_tower, "config"):
            self.vision_dim = getattr(self.vision_tower.config, "hidden_size", 1024)
        
        # Correct for DeepSeek-OCR specific logic if it erroneously picks up LLM dim - bugfix
        if self.vision_dim == 1280:
            print(">>> Correcting detected dimension 1280 -> 1024 for DeepEncoder compatibility.")
            self.vision_dim = 1024

        print(f">>> Vision Dimension: {self.vision_dim}")
        
        self.vision_tower = self.vision_tower.to("cuda")
        del tmp_ds
        gc.collect()

        print(f">>> Loading GOT-OCR Decoder...")
        tmp_got = AutoModel.from_pretrained(
            got_path,
            trust_remote_code=True,
            device_map="cpu",
            torch_dtype=frozen_dtype,
            low_cpu_mem_usage=True
        )

        if hasattr(tmp_got, "language_model"):
            self.decoder = tmp_got.language_model
        else:
            self.decoder = tmp_got.model

        self.decoder_dim = self.decoder.config.hidden_size
        self.decoder = self.decoder.to("cuda")

        self.lm_head = None
        if hasattr(tmp_got, "lm_head"):
            self.lm_head = tmp_got.lm_head
        elif hasattr(self.decoder, "lm_head"):
            self.lm_head = self.decoder.lm_head
        
        if self.lm_head is None:
            vocab_size = self.decoder.config.vocab_size
            self.lm_head = nn.Linear(self.decoder_dim, vocab_size, bias=False)
            if hasattr(self.decoder, "embed_tokens"):
                self.lm_head.weight = self.decoder.embed_tokens.weight
            elif hasattr(self.decoder, "wte"):
                self.lm_head.weight = self.decoder.wte.weight
        
        self.lm_head = self.lm_head.to(dtype=frozen_dtype, device="cuda")
        del tmp_got
        gc.collect()
        torch.cuda.empty_cache()

        # patch - bugfix
        if not hasattr(self.decoder, "prepare_inputs_for_generation"):
            def _prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None,
                                               token_type_ids=None, use_cache=None, **kwargs):
                return {
                    "input_ids": input_ids,
                    "past_key_values": past_key_values,
                    "attention_mask": attention_mask,
                    "token_type_ids": token_type_ids,
                    "use_cache": use_cache,
                    **kwargs
                }
            self.decoder.prepare_inputs_for_generation = types.MethodType(_prepare_inputs_for_generation, self.decoder)
        
        if use_lora:
            print(">>> Injecting LoRA Adapters...")
            lora_config = LoraConfig(
                r=16, 
                lora_alpha=32, 
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], 
                lora_dropout=0.05, 
                bias="none", 
                task_type="CAUSAL_LM"
            )
            self.decoder = get_peft_model(self.decoder, lora_config)
            self.decoder.print_trainable_parameters()

        self.projector = DeepSeekOCRToGOTProjector(self.vision_dim, self.decoder_dim)
        
        self.projector.apply(self._init_weights)
        
        self.img_start_id = tokenizer.convert_tokens_to_ids("<img>") # verified
        self.img_end_id = tokenizer.convert_tokens_to_ids("</img>") # verified

        self.vision_tower.requires_grad_(False)
        self.lm_head.requires_grad_(False)
        self.projector.requires_grad_(True)
        self.decoder.requires_grad_(False)
        self.projector = self.projector.to(device="cuda", dtype=torch.float32)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.01)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)

    def gradient_checkpointing_enable(self, gradient_checkpointing_kwargs=None):
        if hasattr(self.decoder, "gradient_checkpointing_enable"):
            self.decoder.gradient_checkpointing_enable(gradient_checkpointing_kwargs=gradient_checkpointing_kwargs)

    def save_pretrained(self, save_directory):
        if not os.path.exists(save_directory): os.makedirs(save_directory)
        torch.save(self.projector.state_dict(), os.path.join(save_directory, "pytorch_model.bin"))
        with open(os.path.join(save_directory, "config.json"), "w") as f:
            json.dump({
                "vision_dim": self.vision_dim, 
                "decoder_dim": self.decoder_dim,
                "architecture": "DeepSeekOCRToGOTProjector"
            }, f, indent=4)

    def forward(self, pixel_values, input_ids, attention_mask=None, labels=None, **kwargs):
        # apparent bug fix but I'm not so sure
        # kwargs.pop("labels", None)
        
        with torch.no_grad():
            try:
                vision_out = self.vision_tower(pixel_values.to(self.vision_dtype), patch_embeds=None)
            except TypeError:
                vision_out = self.vision_tower(pixel_values.to(self.vision_dtype))
            
            if isinstance(vision_out, (tuple, list)):
                features = vision_out[0]
            elif hasattr(vision_out, "last_hidden_state"):
                features = vision_out.last_hidden_state
            else:
                features = vision_out
            features = features.detach()

        vision_embeds = self.projector(features.to(torch.float32)).to(self.vision_dtype)
        B, N, _ = vision_embeds.shape
        device = vision_embeds.device

        input_emb_fn = self.decoder.get_input_embeddings()
        start_embeds = input_emb_fn(torch.tensor([self.img_start_id], device=device)).expand(B, 1, -1)
        end_embeds = input_emb_fn(torch.tensor([self.img_end_id], device=device)).expand(B, 1, -1)
        text_embeds = input_emb_fn(input_ids)

        inputs_embeds = torch.cat([start_embeds, vision_embeds, end_embeds, text_embeds], dim=1)
        
        vision_len = N + 2
        full_mask = None
        if attention_mask is not None:
            v_mask = torch.ones((B, vision_len), device=device, dtype=attention_mask.dtype)
            full_mask = torch.cat([v_mask, attention_mask], dim=1)

        outputs = self.decoder(
            input_ids=torch.zeros((B, inputs_embeds.shape[1]), device=device, dtype=torch.long),
            inputs_embeds=inputs_embeds,
            attention_mask=full_mask,
            return_dict=True
        )
        
        hidden_states = outputs.last_hidden_state
        relevant_hidden = hidden_states[:, vision_len-1:-1, :].contiguous()
        if self.lm_head is not None:
            logits = self.lm_head(relevant_hidden)
        else:
            raise ValueError("LM Head is not defined.")

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.decoder.config.vocab_size).to(torch.float32), labels.view(-1))

        return CausalLMOutputWithPast(loss=loss, logits=logits if not self.training else None)

# Dataset

In [35]:
class RealTextOCRDataset(Dataset):
    def __init__(self, tokenizer, num_samples=20000):
        self.tokenizer = tokenizer
        self.num_samples = num_samples
        self.fake = Faker('en_US')  # English generator
        
        self.transform = T.Compose([
            T.Resize((1024, 1024), interpolation=T.InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                        std=[0.26862954, 0.26130258, 0.27577711])
        ])

    def __len__(self):
        return self.num_samples

    def generate_content(self):
        # Generate varied content types to make the model robust
        r = random.random()
        if r < 0.4:
            # Type 1: Standard Sentences (The easiest for LLMs)
            return self.fake.sentence(nb_words=10)
        elif r < 0.7:
            # Type 2: Addresses (Structured data)
            return self.fake.address().replace('\n', ', ')
        else:
            # Type 3: Names and Phone numbers
            return f"{self.fake.name()} - {self.fake.phone_number()}"

    def generate_image(self, text):
        # 1. Random Background (White-ish)
        bg_color = random.randint(230, 255)
        img = Image.new('RGB', (1024, 1024), color=(bg_color, bg_color, bg_color))
        draw = ImageDraw.Draw(img)

        # 2. Font Management
        try:
            # Attempt to use a larger, clearer font size
            font_size = random.randint(40, 80) 
            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", font_size)
        except IOError:
            font = ImageFont.load_default()

        # 3. Draw Text (Centered-ish)
        x = random.randint(50, 100)
        y = random.randint(200, 500)
        
        # Simple text wrapping logic
        words = text.split()
        current_line = ""
        for word in words:
            if (len(current_line) + len(word)) * (font_size * 0.5) > 800:
                draw.text((x, y), current_line, fill=(0, 0, 0), font=font)
                y += font_size + 10
                current_line = word + " "
            else:
                current_line += word + " "
        
        # Draw the last line
        draw.text((x, y), current_line, fill=(0, 0, 0), font=font)

        return img

    def __getitem__(self, idx):
        text = self.generate_content()
        image = self.generate_image(text)

        pixel_values = self.transform(image)
        prompt = f"OCR: {text}{self.tokenizer.eos_token}"
        
        # Consistent prefix masking logic from Stage 1 setup
        prefix_enc = self.tokenizer("OCR: ", add_special_tokens=False)
        prefix_len = len(prefix_enc.input_ids)

        encodings = self.tokenizer(
            prompt,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        
        input_ids = encodings.input_ids.squeeze(0)
        attention_mask = encodings.attention_mask.squeeze(0)
        labels = input_ids.clone()
        
        # Mask prefix and padding
        starts_with_bos = (input_ids[0] == self.tokenizer.bos_token_id)
        offset = 1 if starts_with_bos else 0
        labels[:prefix_len + offset] = -100
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Training configuration

In [36]:
DEEPSEEK_OCR_PATH = "deepseek-ai/DeepSeek-OCR"
GOT_PATH = "stepfun-ai/GOT-OCR2_0"
OUTPUT_DIR = "./deepseek_ocr_got_final"

In [37]:
print(">>> 1. Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(GOT_PATH, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(">>> 2. Initializing RealTextOCRDataset...")
train_dataset = RealTextOCRDataset(tokenizer, num_samples=20000)

def collate_fn(batch):
    return {
        "pixel_values": torch.stack([x['pixel_values'] for x in batch]),
        "input_ids": torch.stack([x['input_ids'] for x in batch]),
        "labels": torch.stack([x['labels'] for x in batch]),
        "attention_mask": torch.stack([x['attention_mask'] for x in batch])
    }
    
print(">>> 3. Initializing Fusion Model...")
model = DeepSeekGOTFusion(
    DEEPSEEK_OCR_PATH, 
    GOT_PATH, 
    tokenizer, 
)

>>> 1. Loading Tokenizer...


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


>>> 2. Initializing RealTextOCRDataset...
>>> 3. Initializing Fusion Model...
>>> Loading DeepSeek-OCR Encoder...


Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>> Vision Dimension: 1024
>>> Loading GOT-OCR Decoder...
>>> Injecting LoRA Adapters...
trainable params: 7,569,408 || all params: 568,098,048 || trainable%: 1.3324


In [38]:
MAX_STEPS = 63

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    learning_rate=1e-5,
    max_steps=MAX_STEPS,   # Use max_steps for streaming
    fp16=True,
    gradient_checkpointing=True,
    logging_steps=1,
    save_strategy="steps",
    save_steps=50,      
    remove_unused_columns=False,
    report_to="none",
    save_safetensors=False,
    dataloader_pin_memory=False,
    prediction_loss_only=True,
    max_grad_norm=0.5,
    lr_scheduler_type="cosine",
    warmup_steps=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
)

# Train

In [ ]:
print(">>> 4. Starting Training...")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f">>> Trainable Parameters: {trainable_params:,}")

trainer.train()

print(f">>> 5. Saving Projector to {OUTPUT_DIR}...")
model.save_pretrained(OUTPUT_DIR)
print(">>> Done.")